## Using the Bukdu Webserver Framework

You can use just this code also in a Docker container.

First run the notebook ar.ipynb to create the datafiles in the containers test_sshd and test_sshd2!

In [1]:
using Pkg; Pkg.activate(".")

 Activating environment at `~/projects/TestAppliAR/Project.toml`


In [2]:
using Bukdu, Distributed, DataFrames

### Define the controller

The controller object gives access to the request and response object. E.g `resp = WebController.conn.response` what you need to read the fields of a web form. 

In [3]:
struct WebController <: ApplicationController
    conn::Conn
end

### The Functions

The functions we use in this example are located in the file functions.jl. All function needs the object WebController as argument, except the function template!

- template(t::String), the Bootstrap template.
- index(c::WebController)
- aging_report(c::WebController)

#### Bootstrap template

'Very often, especially on small screens, you want to hide the navigation links and replace them with a button that should reveal them when clicked on.'

See w3schools.com: [Collapsing The Navigation Bar](https://www.w3schools.com/bootstrap4/tryit.asp?filename=trybs_navbar_collapse)

In [4]:
include("functions.jl");

### Define the default route

In [5]:
routes() do
  get("/", WebController, index)
end

"/"

### Connect to the Docker containers test_sshd and test_sshd2

The course [BAWJ](https://www.appligate.nl/BAWJ/stable/chapter13/#.-Creating-SSH-Enabled-Containers) shows how to create the containers. 

In [6]:
addprocs([("rob@172.17.0.2", 1)]; exeflags=`--project=$(Base.active_project())`, tunnel=true, dir="/home/rob")
addprocs([("rob@172.17.0.3", 1)]; exeflags=`--project=$(Base.active_project())`, tunnel=true, dir="/home/rob")
gl_pid = procs()[2] # general ledger
ar_pid = procs()[3] # accounts receivable (invoices/bankstatements)
@everywhere using AppliSales
@everywhere using AppliGeneralLedger
@everywhere using AppliAR
@everywhere using Query

### Retrieve the aging report

In [7]:
ar = @fetchfrom ar_pid report()
ar |> DataFrame

,id_inv,csm,inv_date,amount,days
,String,String,Dates…,Float64,Dates…
1,A1001,Scrooge Investment Bank,"Date(""2021-01-05"")",1210.0,Day(0)


### Show the aging report on your website

In [8]:
routes() do
  get("/agingreport", WebController, aging_report)
end

"/agingreport"

### Start the server

The webserver will listen at port 8004. This port must be opened in Ubuntu.
- sudo ufw -h
- sudo ufw status
- sudo ufw enable
- sudo ufw allow 8004/tcp

After starting the web server go to '127.0.0.1:8004'. You can also use the ip-address (ifconfig) of your computer, e.g. 192.168.1.35:8004.

In [9]:
Bukdu.start(8004, host="0.0.0.0")

Bukdu Listening on 0.0.0.0:8004


Task (runnable) @0x00007f31fc168010

## Utilities

### Stop the webserver

In [13]:
Bukdu.stop()

Bukdu has stopped.


### Display General Ledger Accounts
- 1150 - Bank
- 1300 - Accounts Receivable
- 4000 - VAT
- 8000 - Sales

In [13]:
gl_account_id = 1300

1300

In [14]:
gl = @fetchfrom gl_pid AppliGeneralLedger.read_from_file("./test_ledger.txt")
df = gl |> @filter(_.accountid == gl_account_id) |> DataFrame
df[[:invoice_nbr, :customerid, :debit, :credit, :descr]]

,invoice_nbr,customerid,debit,credit,descr
,String,String,Float64,Float64,String
1,A1001,Scrooge Investment Bank,1210.0,0.0,Learn Smiling
2,A1002,Duck City Chronicals,2420.0,0.0,Learn Smiling
3,A1003,Donalds Hardware Store,1210.0,0.0,Learn Smiling
4,A1002,Duck City Chronicals,0.0,2420.0,Learn Smiling
5,A1003,Donalds Hardware Store,0.0,1210.0,Learn Smiling
